In [ ]:
# Limpeza e Organização de Dados

import pandas as pd # 
import numpy as np
import unicodedata
import re

# Carregar os dados
caminho = r'c:\Users\RODRIGUES\Desktop\ANALISE_EXPLORATORIO\PIPELINE\DADOS_TEMPORAL_2024.csv'
df = pd.read_csv(caminho)

# Função para limpar nomes das colunas
def limpar_nome_coluna(coluna):
    coluna = unicodedata.normalize('NFKD', coluna).encode('ASCII', 'ignore').decode('utf-8')  # Remove acentuação
    coluna = re.sub(r'[^\w\s]', '', coluna)        # Remove símbolos especiais
    coluna = re.sub(r'[\s\-]+', '_', coluna)       # Substitui espaços/traços por "_"
    return coluna.lower()                          # Converte para minúsculas

#  Aplicar limpeza nos nomes das colunas
df.columns = [limpar_nome_coluna(col) for col in df.columns]

# Eliminar colunas irrelevantes (ajuste conforme necessário)
colunas_remover = ['id', 'producao_de_gas_nao_associado_mm3', 'producao_de_agua_m3']
df.drop(columns=[col for col in colunas_remover if col in df.columns], inplace=True)

#  Preencher valores nulos com média ou default
df.fillna({
    'campo_quantitativo': df['campo_quantitativo'].mean() if 'campo_quantitativo' in df.columns else 0,
    'campo_categorico': 'desconhecido'
}, inplace=True)

#  Remover duplicatas
df.drop_duplicates(inplace=True)

#  Corrigir tipos de dados
if 'data' in df.columns:
    df['data'] = pd.to_datetime(df['data'], errors='coerce')
if 'valor' in df.columns:
    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

#  Ordenar e criar colunas derivadas
if 'data' in df.columns:
    df.sort_values(by='data', ascending=True, inplace=True)
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month

#  Agrupamento por categoria (ajuste conforme estrutura)
if 'categoria' in df.columns and 'valor' in df.columns:
    resumo = df.groupby('categoria')['valor'].agg(['mean', 'sum', 'count']).reset_index()
else:
    resumo = pd.DataFrame()

#  Exportar os resultados
df.to_csv('dados_limpos.csv', index=False)
if not resumo.empty:
    resumo.to_csv('resumo_categorico.csv', index=False)

# visualização
print("\n Limpeza, padronização e organização concluídas com sucesso!\n")
print(" Visualização dos registros tratados:\n")
print(df.head())  # Exibe as primeiras 5 linhas do DataFrame


 Limpeza, padronização e organização concluídas com sucesso!

 Visualização dos primeiros registros tratados:

    timestamp  producoes_de_oleo_m3
0  2024/01/01               139.214
1  2024/01/02                 0.000
2  2024/01/03               102.821
3  2024/01/04                76.128
4  2024/01/05                89.816
